In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

#import matplotlib.pyplot as plt

import cProfile, pstats, io
from pstats import SortKey

torch.manual_seed(0)

In [2]:
import numpy as np
ids=get_ids_in_folder(base_path="./data/public/1.0.1/test/")

idx=10
data=load_npz_by_id(idx,base_path="./data/public/1.0.1/test/")



#data=np.load("path")
tmp_name=data["alos"][idx] #shape : 5000;4;4;4;3 (batch / time / h / w / c)
print(tmp_name.shape)

#3456
#11520
#1920
#420
#192



(4, 4, 4, 3)


In [3]:
#create_dataset(name="tiny", mode="train",max_imgs=-1)
#create_dataset(name="tiny", mode="test",max_imgs=-1)
#create_dataset(name="tiny", mode="validation",max_imgs=-1)

In [ ]:
config=read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
transconfig=transformations_config_flair("./data/bands_info/bands.yaml",config)

dataset=CustomPlanted("./data/custom_planted/tiny_train.h5", config=config, trans_config=transconfig)

: 

In [ ]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torch
import os

seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
labels = load_json_to_dict("./data/labels.json")
bands_yaml = "./data/bands_info/bands.yaml"

trans_config = transformations_config_flair(bands_yaml, config_model)
xp_name = "test_Atos"

wand = False
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=get_xp_name(config_model['encoder']) + " modalities",
            project="Atos_modalities",
            config=config_model
        )
        wandb_logger = WandbLogger(project="tiny_modalities")

model = Model(config_model, wand=wand, name=xp_name, labels=labels,transform=trans_config)

data_module = CustomPlantedDataModule(
    "./data/custom_planted/tiny",
    config=config_model,
    trans_config=trans_config,
    batch_size=config_model['dataset']['batchsize'],
)

data_module.setup()


# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/",
    filename="best_model-{epoch:02d}-{val_F1:.4f}",
    monitor="val_F1",
    mode="max",
    save_top_k=1,
    verbose=True,
)

early_stop_callback = EarlyStopping(
    monitor="val_F1",
    min_delta=0.00,
    patience=15,
    verbose=False,
    mode="max"
)



# Trainer
trainer = Trainer(
    use_distributed_sampler=False,
    #strategy="ddp",
    #devices=-1,
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=1,
    accelerator="gpu",
    callbacks=[early_stop_callback, checkpoint_callback],
    default_root_dir="./checkpoints/"
)

pr = cProfile.Profile()
pr.enable()
# Fit the model
trainer.fit(model, datamodule=data_module)

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/hugoriffaud/Documents/PLANTED/checkpoints exists and is not empty.

   | Name                      | Type                      | Params | Mode 
-------------------

tensor([1.6927e+00, 2.9825e+00, 7.2191e-02, 1.6393e-01, 7.9744e-01, 1.1370e+00,
        3.4018e-01, 4.9668e-01, 2.4908e+00, 3.9888e+00, 1.3502e+00, 9.2725e-01,
        7.8254e-01, 2.6841e-01, 6.0148e-01, 1.8241e+00, 8.7463e-01, 4.8420e+00,
        5.9868e+00, 3.2450e+00, 5.4189e+01, 5.1616e+01, 4.0791e+00, 6.4818e+02,
        3.0697e+01, 5.3163e+01, 1.7162e+01, 4.1356e+01, 2.1830e+01, 1.7109e+02,
        1.1623e+02, 5.6458e+01, 7.7842e+01, 2.7182e+02, 2.4783e+02, 1.3482e+03,
        3.0094e+02, 6.6089e+02, 6.2418e+02, 2.2470e+03])
Sanity Checking: |          | 0/? [00:00<?, ?it/s]Validation DataLoader created on rank: 0


/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 14 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train DataLoader created on rank: 0                                        


/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 54/1075701 [00:07<42:18:02,  7.06it/s, v_num=203] 


Detected KeyboardInterrupt, attempting graceful shutdown ...
